# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [5]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and more.
2. **Virtual Assistants**: Generative AI-powered virtual assistants can help businesses automate customer service, provide personalized recommendations, and offer expert advice.
3. **Product Design**: Generative AI can aid in the design of new products by generating 2D and 3D models, prototypes, and even entire product lines.
4. **Marketing and Advertising**: Generative AI can create personalized marketing campaigns, generate ad copy, and optimize advertising content for better engagement.
5. **Data Analysis**: Generative AI can help analyze large datasets, identify patterns, and provide insights to inform business decisions.
6. **Predictive Maintenance**: Generative AI-powered predictive maintenance can help businesses predict equipment failures, 

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can create high-quality content, such as articles, social media posts, product descriptions, and more. This can help businesses streamline their content creation process, reduce costs, and increase productivity.
2. **Image and Video Generation**: Generative AI can generate images and videos that are indistinguishable from those created by humans. This has applications in fields like advertising, marketing, and entertainment, where visual content is crucial.
3. **Chatbots and Virtual Assistants**: Generative AI can power chatbots and virtual assistants, enabling businesses to provide 24/7 customer support, answer frequently asked questions, and route complex inquiries to human representatives.
4. **Predictive Analytics and Forecasting**: Generative AI can analyze large datasets and generate predictions about future trends and patterns. This helps 

## Alternative approach - using OpenAI python library to connect to Ollama

In [7]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generate high-quality content such as articles, social media posts, product descriptions, and even entire books. This can save time and resources for content teams.
2. **Marketing Automation**: Use generative AI to create personalized marketing campaigns, emails, and advertisements tailored to individual customers' preferences.
3. **Product Development**: Employ generative AI to design new products, optimize existing designs, and create prototypes. This can streamline the product development process and reduce costs.
4. **Customer Service Chatbots**: Develop chatbots that use generative AI to provide 24/7 customer support, respond to frequently asked questions, and offer personalized recommendations.
5. **Image and Video Generation**: Use generative AI to create high-quality visual content, such as product images, marketing materials, and promotional videos.
6. **Predictive 

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [8]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
p

In [9]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Okay, so I need to understand what LLMs are based on. From what I've heard before, they're these models that can answer questions and generate text, right? But I'm not exactly sure how they work under the hood. The user asked me to define a few core concepts: neural networks, attention, and transformers. Let me think through each of these.

Starting with neural networks. From what I remember, neural networks are inspired by how the human brain works. They have layers of interconnected nodes, or neurons. Each neuron receives input, processes it, applies an activation function to produce an output, and then passes that output on to the next layer. The key aspect here is deep learning, which involves feeding data through multiple levels of these networks to make predictions. So for LLMs, which are similar, they probably use something akin to a deep neural network, structured into layers with shared weights to capture patterns across different data points.

Next up is attention. I 

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [10]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [11]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"

In [12]:
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [14]:
medium = Website("https://medium.com/design-bootcamp/opening-up-the-design-system-for-everyone-11b9b92f3f75")
print(medium.title)
# print(medium.text)

Opening up the Design System for Everyone — The Federated Model | by Anirudh Ramamohan | Bootcamp | Medium


In [15]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [16]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(medium))

In [18]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [26]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = MODEL,
        messages = messages_for(website),
    )
    return response['message']['content']

In [27]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [28]:
display_summary("https://medium.com/design-bootcamp/opening-up-the-design-system-for-everyone-11b9b92f3f75")

**Summary of the Website**
==========================

### Introduction

The article discusses how to open up a design system for everyone in an organization. The author, Anirudh Ramamohan, shares his experience with creating a process for proposing and implementing changes to the design system.

### Key Points

* Establishing a clear and easy-to-follow process for proposing and implementing changes
* Empowering team members to suggest changes and gather feedback
* Collaboration between designers from different disciplines during the consolidation process
* Alignment with core values of collaboration, clarity, and trust

### Process Overview

1. Prepare a component case using a Figma playground file template
2. Present on a consolidation call to critique and strengthen the proposal
3. Implement changes by renewing the components page in Figma
4. Communicate changes to the business and update relevant documents

### Conclusion

By following this process, everyone has an opportunity to contribute to the design system's evolution while aligning with its core values.

**Author Information**
--------------------

* Name: Anirudh Ramamohan
* Location: Barcelona
* Bio: Turning messy ideas into beautiful products or posting blogs on them!